In [2]:
import polars as pl
from skolegpt_instruct_dataset.utils import return_filter_char_list


df = pl.read_parquet("./data/filtered_dataset.parquet")

In [86]:
from skolegpt_instruct_dataset.utils import sample_and_print_example

sample_and_print_example(df)

ID
cot.52598

SYSTEM PROMPT:

You are an AI assistant that helps people find information.

QUESTION:

Written and performed by Ben Gibbard, it is an acoustic solo ballad,
and was recorded in monaural with a single microphone and little
editing.  Based on this passage, who wrote i will follow you into the
dark? Let's think fast.

RESPONSE:

Ben Gibbard wrote "I Will Follow You Into the Dark".


id,system_prompt,question,response,source
str,str,str,str,str
"""cot.52598""","""You are an AI …","""Written and pe…","""Ben Gibbard wr…","""cot"""


In [87]:
df_raw = pl.read_parquet("./data/sampled_dataset.parquet")

In [152]:
sample_and_print_example(df_raw)

ID
t0.595983

SYSTEM PROMPT:

You are an AI assistant. Provide a detailed answer so user don’t need
to search outside to understand the answer.

QUESTION:

Question: Information:  - In elementary geometry, a polytope is a
geometric object with flat sides, and may exist in any general number
of dimensions "n" as an "n"-dimensional polytope or "n"-polytope. For
example, a two-dimensional polygon is a 2-polytope and a three-
dimensional polyhedron is a 3-polytope.  - In computer science,
program optimization or software optimization is the process of
modifying a software system to make some aspect of it work more
efficiently or use fewer resources. In general, a computer program may
be optimized so that it executes more rapidly, or is capable of
operating with less memory storage or other resources, or draw less
power.  - In computer science and particularly in compiler design,
loop nest optimization (LNO) is an optimization technique that applies
a set of loop transformations for the pur

id,system_prompt,question,response,source
str,str,str,str,str
"""t0.595983""","""You are an AI …","""Question: Info…","""The polytope m…","""t0"""
